In [1]:
import subprocess
import importlib

required_libraries = [
    'torch', 'pyaudio', 'wave', 'numpy', 
    'transformers', 'scipy'
]

conda_command = 'conda install'
pip_command = 'pip install'

for library in required_libraries:
    try:
        importlib.import_module(library)
    except ImportError:
        print(f'{library} not found. Installing...')
        if 'conda' in subprocess.run(['which', 'python'], capture_output=True, text=True).stdout:
            subprocess.run([conda_command, library, '-y'])
        else:
            subprocess.run([pip_command, library])

print('All required libraries are installed.')


All required libraries are installed.


In [9]:
import                                  torch
import                                  pyaudio
import                                  wave
import                                  numpy                                               as  np
from    transformers            import  WhisperProcessor, WhisperForConditionalGeneration
from    scipy.io                import  wavfile
from    transformers            import  pipeline
# Parameters for audio input
FORMAT = pyaudio.paInt16   # Format of audio samples
CHANNELS = 1               # Number of audio channels (1 for mono, 2 for stereo)
RATE = 16000               # Sampling rate (samples per second)
CHUNK = 1024               # Number of frames per buffer


In [ ]:
import subprocess

def encode_to_x265(input_file, output_file):
    """
    Encode a video file to x265 format using FFmpeg.

    :param input_file: Input video file path.
    :param output_file: Output file path for the encoded video.
    """
    # Use FFmpeg with libx265 to encode the video file
    ffmpeg_cmd = f'ffmpeg -i  "{input_file}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{output_file}"'
    subprocess.run(ffmpeg_cmd, shell=True)

def write_text_to_file(filename, text):
    """
    Write a text string to a file.

    :param filename: The name of the file to write to.
    :param text: The text string to write to the file.
    """
    try:
        with open(filename, 'w') as file:
            file.write(text)
        print(f"Text has been written to {filename}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [2]:
def save_wave_file(cached_audio, p, AMPLIFICATION_FACTOR=10, output_file="cached_audio.wav"):
    """
    Save the cached audio data to a WAV file.

    Args:
        cached_audio (numpy.ndarray): The audio data to be saved.
        p (pyaudio.PyAudio): PyAudio instance.
        AMPLIFICATION_FACTOR (int, optional): Amplification factor for the audio. Default is 10.
        output_file (str, optional): Output file path. Default is "cached_audio.wav".
    """
    # Save the cached audio to a WAV file
    with wave.open(output_file, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes((cached_audio * AMPLIFICATION_FACTOR).tobytes())
    print(f"Saved cached audio to {output_file}")


def record_audio():
    """
    Record audio from the microphone.

    Returns:
        tuple: Numpy array of recorded audio and PyAudio instance.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    # Initialize an empty list to store audio data
    audio_cache = []
    try:
        print("Recording... (Press Ctrl+C to stop)")
        while True:
            # Read audio data from the microphone
            data = stream.read(CHUNK)

            # Convert the binary audio data to a numpy array
            audio_sample = np.frombuffer(data, dtype=np.int16)

            # Append the audio sample to the cache
            audio_cache.append(audio_sample)

    except KeyboardInterrupt:
        print("Recording stopped.")

    # Close the audio stream and PyAudio instance
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Convert the list of audio samples into a numpy array
    return np.concatenate(audio_cache), p


# 1.Audio Extact

In [ ]:
'ffmpeg -i ./Conv_1.\ Course\ content_720p_x265.mp4 -vn -acodec pcm_s16le -ar 16000 -ac 1  ./output.wav'

# 2.Audio-to-text

In [3]:
# https://huggingface.co/openai/whisper-large-v2
# https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-english

index = -1
modelss = [
            # "openai/whisper-tiny",   #39m params useless
            "openai/whisper-base",   #74m params
            "openai/whisper-small",  #244m params very well
            "openai/whisper-medium", #769m params
           ]
# load model and processor
processor = WhisperProcessor.from_pretrained(modelss[index])
model = WhisperForConditionalGeneration.from_pretrained(modelss[index]).to('cuda')
model.config.forced_decoder_ids = None


In [16]:
# audio_cache,p = record_audio()
# save_wave_file(audio_cache,p)

ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map


Recording... (Press Ctrl+C to stop)
Recording stopped.
Saved cached audio to cached_audio.wav


In [4]:
samplerat1, data1 = wavfile.read('output.wav')

In [5]:
data1= data1/10

In [ ]:
# hass memory of 27 seconds

In [6]:
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")
text = ''

S_rate = 16000
for index in range(len(data1)//(25*S_rate)):
    input_features = processor(data1[(S_rate*index*25):(S_rate*(index+1)*25)], sampling_rate=S_rate, return_tensors="pt").input_features.to('cuda')
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    text += transcription[0] + ' '


/home/ysn/anaconda3/envs/torch3.9/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
text

" Hello and welcome to the course, Generative Adversary Network School with guides. My name is Jones, I am the founder of the AI Experts Academy, which is an internship online platform with a lot of courses on artificial intelligence and machine learning.  Hello, I'm going to teach all the lectures of this course. Now I'm going to show you the contents that will be covered. First, you are going to learn some interesting applications of games, as well as a brief introduction to the intuition of how it works. Then we are going to move on.  The practical implementations. The first two implementations that you are going to learn are DC Game and W Game. They are the same placed ones. I have opened the first implementation about Deep Convolutional Games. As you can see, all implementations will be...  Don't step by step using Google Cloud online, so you don't need to worry about installing all the libraries on your machine. Another advantage about using Google Cloud is that it allows you to 

In [7]:
' '.join(text.split(' ')[0:(total_lenght//3)]) 

NameError: name 'total_lenght' is not defined

In [10]:
import re


In [21]:

# Define the regular expression pattern
paragraph_pattern = r'\.'

# Your long transcript
transcript = text

# Split the transcript into paragraphs
paragraphs = re.split(paragraph_pattern, transcript)

# Optionally, strip leading/trailing whitespace
paragraphs = [p.strip() for p in paragraphs]


# Using a list comprehension to remove empty strings
filtered_list = [item for item in paragraphs if item.strip() != ""]

print(filtered_list)


['Hello and welcome to the course, Generative Adversary Network School with guides', 'My name is Jones, I am the founder of the AI Experts Academy, which is an internship online platform with a lot of courses on artificial intelligence and machine learning', "Hello, I'm going to teach all the lectures of this course", "Now I'm going to show you the contents that will be covered", 'First, you are going to learn some interesting applications of games, as well as a brief introduction to the intuition of how it works', 'Then we are going to move on', 'The practical implementations', 'The first two implementations that you are going to learn are DC Game and W Game', 'They are the same placed ones', 'I have opened the first implementation about Deep Convolutional Games', 'As you can see, all implementations will be', "Don't step by step using Google Cloud online, so you don't need to worry about installing all the libraries on your machine", 'Another advantage about using Google Cloud is tha

# 3.Summurazation

In [19]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

"Hello and welcome to the course, Generative Adversary Network School with guides My name is Jones, I am the founder of the AI Experts Academy, which is an internship online platform with a lot of courses on artificial intelligence and machine learning Hello, I'm going to teach all the lectures of this course Now I'm going to show you the contents that will be covered First, you are going to learn some interesting applications of games, as well as a brief introduction to the intuition of how it works Then we are going to move on The practical implementations The first two implementations that you are going to learn are DC Game and W Game They are the same placed ones I have opened the first implementation about Deep Convolutional Games As you can see, all implementations will be Don't step by step using Google Cloud online, so you don't need to worry about installing all the libraries on your machine Another advantage about using Google Cloud is that it allows you to use a GPU, so the 

In [24]:
print(summarizer(' '.join(filtered_list[0:(len(filtered_list)//3)]) , min_length=90, do_sample=True))


[{'summary_text': "The course is called Generative Adversary Network School with guides. The first project is the most classic, and if you already ever taken a course on this subject, probably you have implemented this project. All implementations will be Don't step by step using Google Cloud online, so you don't need to worry about installing all the libraries on your machine. Using Google Cloud is that it allows you to use a GPU, so the training process of the game will be faster."}]
